<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
        Import Needed Libraries
    </h1>
    </div>

In [1138]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.offline import iplot , plot
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler , LabelEncoder
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor , BaggingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error , r2_score
import warnings
warnings.filterwarnings('ignore')

<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
        Reading Dataset And Get Info
    </h1>
    </div>

## Load Data

In [1139]:
df = pd.read_csv('Car details v3.csv')

In [1140]:
# Show Sample of Data
df.sample(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
3832,Lexus ES 300h,2019,5150000,20000,Petrol,Dealer,Automatic,First Owner,22.37 kmpl,2487 CC,214.56 bhp,202Nm@ 3600-5200rpm,5.0
282,Hyundai Grand i10 AT Asta,2015,465000,63063,Petrol,Dealer,Automatic,First Owner,18.9 kmpl,1197 CC,82 bhp,114Nm@ 4000rpm,5.0
4608,Volkswagen Vento Diesel Highline,2011,340000,90000,Diesel,Individual,Manual,First Owner,20.54 kmpl,1598 CC,103.6 bhp,250Nm@ 1500-2500rpm,5.0
2120,Maruti Swift Dzire LDi,2010,215000,120000,Diesel,Individual,Manual,Third Owner,19.3 kmpl,1248 CC,73.9 bhp,190Nm@ 2000rpm,5.0
4421,Maruti Baleno Alpha 1.3,2018,740000,38817,Diesel,Dealer,Manual,First Owner,27.39 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0


In [1141]:
# Show Shape of Data
print(f"Number of Row : {df.shape[0]}\nNumber of Columns : {df.shape[1]}")

Number of Row : 8128
Number of Columns : 13


In [1142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [1143]:
# Check NaN Value
df.isna().sum()

name               0
year               0
selling_price      0
km_driven          0
fuel               0
seller_type        0
transmission       0
owner              0
mileage          221
engine           221
max_power        215
torque           222
seats            221
dtype: int64

In [1144]:
# Describe Numiric Data
df.describe()

,year,selling_price,km_driven,seats
count,8128.000000,8.128000e+03,8.128000e+03,7907.000000
mean,2013.804011,6.382718e+05,6.981951e+04,5.416719
std,4.044249,8.062534e+05,5.655055e+04,0.959588
min,1983.000000,2.999900e+04,1.000000e+00,2.000000
25%,2011.000000,2.549990e+05,3.500000e+04,5.000000
50%,2015.000000,4.500000e+05,6.000000e+04,5.000000
75%,2017.000000,6.750000e+05,9.800000e+04,5.000000
max,2020.000000,1.000000e+07,2.360457e+06,14.000000


In [1145]:
# Describe non Numiric 
df.describe(exclude=np.number)

,name,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque
count,8128,8128,8128,8128,8128,7907,7907,7913,7906
unique,2058,4,3,2,5,393,121,322,441
top,Maruti Swift Dzire VDI,Diesel,Individual,Manual,First Owner,18.9 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm
freq,129,4402,6766,7078,5289,225,1017,377,530


In [1146]:
pd.DataFrame({'Count':df.shape[0],
              'Null':df.isnull().sum(),
              'Null %':df.isnull().mean() * 100,
              'Cardinality':df.nunique()
})

,Count,Null,Null %,Cardinality
name,8128,0,0.000000,2058
year,8128,0,0.000000,29
selling_price,8128,0,0.000000,677
km_driven,8128,0,0.000000,921
fuel,8128,0,0.000000,4
seller_type,8128,0,0.000000,3
transmission,8128,0,0.000000,2
owner,8128,0,0.000000,5
mileage,8128,221,2.718996,393
engine,8128,221,2.718996,121


<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
        Handling Missing Data (Nulls)
    </h1>
    </div>

## Handling Missing Data

### The number of missing values ​​is small and can be ignored. (Maximum 225 from 8128 Rows)

In [1147]:
# Delete Nulls
df.dropna(inplace=True , ignore_index=True)

In [1148]:
# Data After Delete Nulls
df.sample(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
2482,Maruti Alto 800 LXI,2017,310000,20000,Petrol,Individual,Manual,First Owner,24.7 kmpl,796 CC,47.3 bhp,69Nm@ 3500rpm,5.0
3754,Toyota Fortuner 4x2 AT,2015,2000000,60000,Diesel,Individual,Automatic,First Owner,12.55 kmpl,2982 CC,168.5 bhp,360Nm@ 1400-3200rpm,7.0
3861,Mahindra XUV500 W6 2WD,2014,750000,80000,Diesel,Individual,Manual,Second Owner,15.1 kmpl,2179 CC,140 bhp,330Nm@ 1600-2800rpm,7.0
6788,Hyundai Verna CRDi SX ABS,2008,150000,120000,Diesel,Individual,Manual,Second Owner,18.1 kmpl,1493 CC,110 bhp,"24@ 1,900-2,750(kgm@ rpm)",5.0
5627,Mahindra Scorpio SLE BSIII,2013,550000,80000,Diesel,Individual,Manual,Second Owner,12.05 kmpl,2179 CC,120 bhp,290Nm@ 1800-2800rpm,8.0


In [1149]:
# The New of Shape Data
print(f"Number of Row : {df.shape[0]}\nNumber of Columns : {df.shape[1]}")

Number of Row : 7906
Number of Columns : 13


In [1150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7906 entries, 0 to 7905
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           7906 non-null   object 
 1   year           7906 non-null   int64  
 2   selling_price  7906 non-null   int64  
 3   km_driven      7906 non-null   int64  
 4   fuel           7906 non-null   object 
 5   seller_type    7906 non-null   object 
 6   transmission   7906 non-null   object 
 7   owner          7906 non-null   object 
 8   mileage        7906 non-null   object 
 9   engine         7906 non-null   object 
 10  max_power      7906 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7906 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 803.1+ KB


In [1151]:
# Check NaN Value
df.isna().sum()

name             0
year             0
selling_price    0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
mileage          0
engine           0
max_power        0
torque           0
seats            0
dtype: int64

In [1152]:
# Describe Numiric (After Delete Null)
df.describe()

,year,selling_price,km_driven,seats
count,7906.000000,7.906000e+03,7.906000e+03,7906.000000
mean,2013.983936,6.498137e+05,6.918866e+04,5.416393
std,3.863695,8.135827e+05,5.679230e+04,0.959208
min,1994.000000,2.999900e+04,1.000000e+00,2.000000
25%,2012.000000,2.700000e+05,3.500000e+04,5.000000
50%,2015.000000,4.500000e+05,6.000000e+04,5.000000
75%,2017.000000,6.900000e+05,9.542500e+04,5.000000
max,2020.000000,1.000000e+07,2.360457e+06,14.000000


In [1153]:
# Describe non Numiric (After Delete Null)
df.describe(exclude=np.number)

,name,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque
count,7906,7906,7906,7906,7906,7906,7906,7906,7906
unique,1982,4,3,2,5,393,121,320,441
top,Maruti Swift Dzire VDI,Diesel,Individual,Manual,First Owner,18.9 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm
freq,129,4299,6563,6865,5215,225,1017,377,530


In [1154]:
# The New Cardinality
pd.DataFrame({'Count':df.shape[0],
              'Null':df.isnull().sum(),
              'Null %':df.isnull().mean() * 100,
              'Cardinality':df.nunique()
})

,Count,Null,Null %,Cardinality
name,7906,0,0.0,1982
year,7906,0,0.0,27
selling_price,7906,0,0.0,670
km_driven,7906,0,0.0,898
fuel,7906,0,0.0,4
seller_type,7906,0,0.0,3
transmission,7906,0,0.0,2
owner,7906,0,0.0,5
mileage,7906,0,0.0,393
engine,7906,0,0.0,121


<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
         EDA of Data and Get Insights
    </h1>
    </div>

## Types of Car Brands

In [1155]:
Brands_of_car = df['name'].tolist()
for i in range(len(Brands_of_car)):
    car = Brands_of_car[i].split(' ')
    Brands_of_car[i] = car[0]
Brands_of_car = pd.Series(Brands_of_car)

In [1156]:
# Replace Model of each car to only Name of Brand
df['name'] = Brands_of_car

In [1157]:
print(f"Number of Unique Brand of Car {df['name'].nunique()}")

Number of Unique Brand of Car 31


In [1158]:
Count_of_Brands_Car = df['name'].value_counts()[:20]
px.bar(Count_of_Brands_Car,
       title = 'Top 20 Brands of Car',
       template = 'plotly_dark',
       color = Count_of_Brands_Car,
       labels={'value':'Number of Frequency in Data','name':'Brand of Car'},
       text_auto=True
       )

## Last five years of car Production

In [1159]:
brand_production_per_Year = df.groupby('year')['name']

In [1160]:
print(f"Number of Cars Production in 2020 only '{len(brand_production_per_Year.get_group(2020))}'")

Number of Cars Production in 2020 only '74'


Ignore 2020 Because This year not complete only 74 car !

In [1161]:
# Get Years from 2015 to 2019
last_five_years = list(df['year'].value_counts().sort_index(ascending=False)[1:6].index)
last_five_years.sort()

In [1162]:
for year in last_five_years:
  iplot(px.bar(brand_production_per_Year.get_group(year).value_counts()[:10],
       title = f'Top 10 Brands of Car in {year}',
       template = 'plotly_dark',
       color = brand_production_per_Year.get_group(year).value_counts()[:10],
       labels={'value':f'Number of Cars Produced in {year}','name':'Brand of Car'},
       text_auto=True
       ))

## Total Selling Price per Years

In [1163]:
sum_Price_Years = df.groupby('year')['selling_price'].sum()
px.line(sum_Price_Years,
        x = sum_Price_Years.index,
        y = sum_Price_Years.values,
        labels={'y':'Total Selling Price','year':'Year of Selling the Car'},
        template = 'plotly_dark',
        markers = True,
        line_shape = 'spline',
        color_discrete_sequence = ['red'],
        log_x = True)

In [1164]:
print(f"Best year for total car sales profit in the Market in '{sum_Price_Years.idxmax()}' is ${sum_Price_Years.values.max():,.2f}")

Best year for total car sales profit in the Market in '2019' is $1,035,982,984.00


## Does the type of fuel affect on the Selling Price?

In [1165]:
type_of_fuel = df['fuel'].value_counts()
px.pie(title='Types of Fuel',
       values = type_of_fuel.values,
       names = type_of_fuel.index,
       template = 'plotly_dark'
       ).update_traces(textinfo='label+percent')

In [1166]:
price_Type_Fuel = df.groupby('fuel')['selling_price'].mean()
px.bar(x = price_Type_Fuel.index,
       y = price_Type_Fuel.values,
       template = 'plotly_dark',
       color = price_Type_Fuel.index,
       text_auto = True,
       title = 'Fuel type affects selling Price',
       labels = {'y':'Total Selling Price','x':'Type of Fuel'}
       )

- Diesel and Petrol fuel are the most popluar in the data
- price goes higher with Diesel fuel

## Which is better transmission ,Type of owner and Type Seller Type for the selling price?

### First -> Type of transmission on Selling Price

In [1167]:
type_transmission = df['transmission'].value_counts()
px.pie(title='Types of Transmission',
       values = type_transmission.values,
       names = type_transmission.index,
       template = 'plotly_dark'
       ).update_traces(textinfo='label+percent')

- Manual Cars are most popular tham automatic cars

In [1168]:
Selling_price_transmission = df.groupby('transmission')['selling_price'].mean()
px.bar(x = Selling_price_transmission.index,
       y = Selling_price_transmission.values,
       template = 'plotly_dark',
       color = Selling_price_transmission.index,
       text_auto = True,
       title = 'Transmission type affects selling Price',
       labels = {'y':'Total Selling Price','x':'Type of Transmission'}
       )

### Second -> Type of Seller on Selling Price

In [1169]:
seller_types = df['seller_type'].value_counts()
px.pie(title = 'Seller type affects selling Price',
       values = seller_types.values,
       names = seller_types.index,
       template = 'plotly_dark'
       ).update_traces(textinfo='label+percent')

- Individual sellers are most popular than other sellers

In [1170]:
Selling_price_seller_types = df.groupby('seller_type')['selling_price'].mean()
px.bar(x = Selling_price_seller_types.index,
       y = Selling_price_seller_types.values,
       template = 'plotly_dark',
       color = Selling_price_seller_types.index,
       text_auto = True,
       title = 'Seller type affects selling Price',
       labels = {'y':'Total Selling Price','x':'Type of Seller'}
       )

### Third -> Type of Owner on Selling Price

In [1171]:
owner_types = df['owner'].value_counts()
px.pie(title = 'Seller type affects selling Price',
       values = owner_types.values,
       names = owner_types.index,
       template = 'plotly_dark'
       ).update_traces(textinfo='label+percent')

- First Owner are most popular than other Owners

In [1172]:
Selling_price_owner_types = df.groupby('owner')['selling_price'].mean()
px.bar(x = Selling_price_owner_types.index,
       y = Selling_price_owner_types.values,
       template = 'plotly_dark',
       color = Selling_price_owner_types.index,
       text_auto = True,
       title = 'Owner type affects selling Price',
       labels = {'y':'Total Selling Price','x':'Type of Owner'}
       )

## Total Insights :
- Diesel and Petrol fuel are the most popluar in the data
- Individual sellers are most popular than other sellers
- First Owner are most popular than other Owners
- Manual Cars are most popular tham automatic cars
- price goes higher with Diesel fuel, automatic transmission, Dealer Seller and test Drive Car

<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
         Data Preprocessing
    </h1>
    </div>

## Handling Object DataType

### First Remove any object after numiric data

In [1173]:
df = df.replace({'mileage':'[A-Za-z/]','engine':'[A-Za-z]','max_power':'[A-Za-z]'},'',regex=True)
df['mileage'] = df['mileage'].astype(float)
df['engine'] = df['engine'].astype(float)
df['max_power'] = df['max_power'].astype(float)

### Delete the column 'torque' because there are many words and specifications that can be ignored.

In [1174]:
df.drop('torque' , axis = 1 , inplace = True)

### Change column year to Age of production Car

In [1175]:
df['year'] = 2025 - df['year']
df.rename(columns={'year':'age'},inplace=True)

In [1176]:
# Show Data After some of Preprocessing
df.head()

,name,age,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
0,Maruti,11,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,5.0
1,Skoda,11,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0
2,Honda,19,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,5.0
3,Hyundai,15,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,5.0
4,Maruti,18,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,5.0


In [1177]:
# After Change DataType of Features
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7906 entries, 0 to 7905
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           7906 non-null   object 
 1   age            7906 non-null   int64  
 2   selling_price  7906 non-null   int64  
 3   km_driven      7906 non-null   int64  
 4   fuel           7906 non-null   object 
 5   seller_type    7906 non-null   object 
 6   transmission   7906 non-null   object 
 7   owner          7906 non-null   object 
 8   mileage        7906 non-null   float64
 9   engine         7906 non-null   float64
 10  max_power      7906 non-null   float64
 11  seats          7906 non-null   float64
dtypes: float64(4), int64(3), object(5)
memory usage: 741.3+ KB


### Encode Object DataType 

In [1178]:
column_to_encode = ['name','fuel','seller_type','transmission','owner']
le = LabelEncoder()
for column in column_to_encode:
    df[column] = le.fit_transform(df[column])

In [1179]:
# Show Data After some of Encode
df.head()

,name,age,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
0,20,11,450000,145500,1,1,1,0,23.40,1248.0,74.00,5.0
1,26,11,370000,120000,1,1,1,2,21.14,1498.0,103.52,5.0
2,10,19,158000,140000,3,1,1,4,17.70,1497.0,78.00,5.0
3,11,15,225000,127000,1,1,1,0,23.00,1396.0,90.00,5.0
4,20,18,130000,120000,3,1,1,0,16.10,1298.0,88.20,5.0


In [1180]:
# Final Info after Encode
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7906 entries, 0 to 7905
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           7906 non-null   int32  
 1   age            7906 non-null   int64  
 2   selling_price  7906 non-null   int64  
 3   km_driven      7906 non-null   int64  
 4   fuel           7906 non-null   int32  
 5   seller_type    7906 non-null   int32  
 6   transmission   7906 non-null   int32  
 7   owner          7906 non-null   int32  
 8   mileage        7906 non-null   float64
 9   engine         7906 non-null   float64
 10  max_power      7906 non-null   float64
 11  seats          7906 non-null   float64
dtypes: float64(4), int32(5), int64(3)
memory usage: 586.9 KB


## Assign feature and target variables

In [1181]:
X = df.drop('selling_price' , axis = 1)
y = df['selling_price']

## Splitting Data 

In [1182]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.25 , random_state = 44 , shuffle = True)

In [1183]:
print(f'Shape of X_Train {X_train.shape}')
print(f'Shape of X_Test {X_test.shape}')
print(f'Shape of Y_Train {y_train.shape}')
print(f'Shape of Y_Test {y_test.shape}')

Shape of X_Train (5929, 11)
Shape of X_Test (1977, 11)
Shape of Y_Train (5929,)
Shape of Y_Test (1977,)


<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
         Modeling
    </h1>
    </div>

## Use Different Models Algorithm

In [1184]:
def Kfold(model,model_name):
    model = cross_val_score(model , X , y , cv = 10)
    model_score = np.average(model)
    print(f"{model_name} score on cross validation: {model_score * 100}%")

def train(model,model_name):
    model.fit(X_train,y_train)
    model_train_score = model.score(X_train,y_train)
    model_test_score = model.score(X_test,y_test)
    print(f"{model_name} model score on Training data: {model_train_score * 100}%\n{model_name} model score on Testing data: {model_test_score * 100}%")

def r2(model,model_name):
    score = r2_score(y_test , model.predict(X_test))
    print(f"R2 Score for {model_name} is {score * 100}%")

## Linear Regression Model

In [1185]:
lr = LinearRegression()
Kfold(lr,'Linear Regression')
train(lr,'Linear Regression')
r2(lr,'Linear Regression')

Linear Regression score on cross validation: 66.00283903822167%
Linear Regression model score on Training data: 67.61174801974936%
Linear Regression model score on Testing data: 68.83581706676102%
R2 Score for Linear Regression is 68.83581706676102%


## Random Forest Model

In [1186]:
rf_model = RandomForestRegressor(n_estimators = 100 , max_depth = 10)
Kfold(rf_model, "Random Forest")
train(rf_model, "Random Forest")
r2(rf_model, "Random Forest")

Random Forest score on cross validation: 96.29379205223591%
Random Forest model score on Training data: 98.89186710222864%
Random Forest model score on Testing data: 97.10260117257225%
R2 Score for Random Forest is 97.10260117257225%


## Bagging Model

In [1187]:
bag = BaggingRegressor(n_estimators= 10)
Kfold(bag,'Bagging')
train(bag,'Bagging')
r2(bag,'Bagging')

Bagging score on cross validation: 96.31928791835375%
Bagging model score on Training data: 99.21227183683885%
Bagging model score on Testing data: 96.83661107474465%
R2 Score for Bagging is 96.83661107474465%


## Gradient Boosting Model

In [1188]:
GrBoost = GradientBoostingRegressor()
Kfold(GrBoost,'Gradient Boosting')
train(GrBoost,'Gradient Boosting')
r2(GrBoost,'Gradient Boosting')

Gradient Boosting score on cross validation: 95.42035854900504%
Gradient Boosting model score on Training data: 97.45946904753747%
Gradient Boosting model score on Testing data: 96.3359637563581%
R2 Score for Gradient Boosting is 96.3359637563581%


## KNN

In [1189]:
knn = KNeighborsRegressor()
Kfold(knn,'KNN')
train(knn,'KNN')
r2(knn,'KNN')

KNN score on cross validation: 79.96173785489884%
KNN model score on Training data: 86.79608828404307%
KNN model score on Testing data: 82.55109300514209%
R2 Score for KNN is 82.55109300514209%


## Decision Tree Regressor

In [1190]:
DTR = DecisionTreeRegressor()
Kfold(DTR,'Decision Tree')
train(DTR,'Decision Tree')
r2(DTR,'Decision Tree')

Decision Tree score on cross validation: 94.47277508453757%
Decision Tree model score on Training data: 99.9662127253933%
Decision Tree model score on Testing data: 96.18699843091021%
R2 Score for Decision Tree is 96.18699843091021%


## XGBoost

In [1191]:
xgboost = XGBRegressor()
Kfold(xgboost, "XGBoost")
train(xgboost, "XGBoost")
r2(xgboost, "XGBoost")

XGBoost score on cross validation: 96.26465590866546%
XGBoost model score on Training data: 99.61505280210022%
XGBoost model score on Testing data: 97.41086015985965%
R2 Score for XGBoost is 97.41086015985965%


- Best Model is XGBoost with accuracy 97.4%

## Save XGBoost Model 

In [1192]:
joblib.dump(xgboost,'XGBoost.sav')

['XGBoost.sav']